# Chapter 5: Elliptic Curves

## Setup

In [2]:
:dep curvelib = { git = "https://github.com/Tranduy1dol/curvelib", version = "0.1.0" }
:dep anyhow = "1.0"

## Utils Implementation
The following generic `EllipticCurve` implementation is inlined here for the exercises. It corresponds to the code previously in `utils/src/elliptic_curve_operations.rs`.

In [ ]:
// Extended GCD
pub fn extended_gcd(x: i64, y: i64) -> (i64, i64, i64) {
    let (mut old_r, mut r) = (x, y);
    let (mut old_s, mut s) = (1, 0);
    let (mut old_t, mut t) = (0, 1);

    while r != 0 {
        let quotient = old_r / r;
        (old_r, r) = (r, old_r - quotient * r);
        (old_s, s) = (s, old_s - quotient * s);
        (old_t, t) = (t, old_t - quotient * t);
    }
    (old_s, old_t, old_r)
}

// Modular Arithmetic
#[derive(Debug, Copy, Clone)]
pub struct Modular(pub i64);

impl Modular {
    pub fn add(&self, a: i64, b: i64) -> i64 {
        ((a + b) % self.0 + self.0) % self.0
    }
    pub fn sub(&self, a: i64, b: i64) -> i64 {
        self.add(a, self.neg(b))
    }
    pub fn mul(&self, a: i64, b: i64) -> i64 {
        ((a * b) % self.0 + self.0) % self.0
    }
    pub fn neg(&self, a: i64) -> i64 {
        (((self.0 - a) % self.0) + self.0) % self.0
    }
    pub fn inv(&self, a: i64) -> i64 {
        let (inv, _, _) = extended_gcd(a, self.0);
        self.add(inv, self.0)
    }
    pub fn div(&self, a: i64, b: i64) -> anyhow::Result<i64> {
        if b == 0 {
            anyhow::bail!("Division by zero");
        } else {
            Ok(self.mul(a, self.inv(b)))
        }
    }
}

// Elliptic Curve
#[derive(Debug, Copy, Clone, Eq)]
pub struct Point {
    pub x: i64,
    pub y: i64,
    pub z: i64,
}

pub enum CurveForm {
    ShortWeierstrass,
    Montgomery,
    TwistedEdwards,
}

pub struct EllipticCurve {
    form: CurveForm,
    field: Modular,
    a: i64,
    b: i64,
}

impl Point {
    pub fn new(x: i64, y: i64, z: i64) -> Point {
        if z == 1 {
            Point { x, y, z }
        } else {
            Point { x: 0, y: 1, z: 0 }
        }
    }
    pub fn is_infinity(&self) -> bool {
        self.x == 0 && self.y == 1 && self.z == 0
    }
}

impl PartialEq<Point> for Point {
    fn eq(&self, other: &Point) -> bool {
        self.x == other.x && self.y == other.y && self.z == other.z
    }
}

impl EllipticCurve {
    pub fn new(form: CurveForm, field: Modular, a: i64, b: i64) -> Self {
        EllipticCurve { form, field, a, b }
    }

    pub fn is_point_on_curve(&self, point: Point) -> bool {
        if point.z == 0 {
            return true;
        }
        match self.form {
            CurveForm::ShortWeierstrass => {
                (point.y * point.y) % self.field.0
                    == (point.x * point.x * point.x + self.a * point.x + self.b) % self.field.0
            }
            _ => false, // Only SW needed for exercises for now
        }
    }

    pub fn inverse(&self, point: Point) -> anyhow::Result<Point> {
        if self.is_point_on_curve(point) {
            Ok(Point {
                x: point.x,
                y: self.field.neg(point.y),
                z: point.z,
            })
        } else {
            anyhow::bail!("Cannot find inverse of this point")
        }
    }

    pub fn projective_add(&self, point_1: Point, point_2: Point) -> anyhow::Result<Point> {
        let field = self.field;
        if point_1.is_infinity() { return Ok(point_2); }
        if point_2.is_infinity() { return Ok(point_1); }

        let u_1 = point_1.z * point_2.y;
        let u_2 = point_1.y * point_2.z;
        let v_1 = point_1.z * point_2.x;
        let v_2 = point_1.x * point_2.z;

        if v_1 == v_2 {
            if u_1 != u_2 { Ok(Point::new(0, 1, 0)) }
            else if point_1.y == 0 { Ok(Point::new(0, 1, 0)) }
            else {
                let w = self.a * point_1.z * point_1.z + 3 * point_1.x * point_1.x;
                let s = point_1.y * point_1.z;
                let b = point_1.x * point_1.y * s;
                let h = w * w - 8 * b;
                let x = 2 * h * s;
                let y = w * (4 * b - h) - 8 * point_1.y * point_1.y * s * s;
                let z = 8 * s * s * s;
                Ok(Point::new(field.div(x, z)?, field.div(y, z)?, field.div(z, z)?))
            }
        } else {
            let u = u_1 - u_2;
            let v = v_1 - v_2;
            let w = point_1.z * point_2.z;
            let a = u * u * w - v * v * v - 2 * v * v * v_2;
            let x = v * a;
            let y = u * (v * v * v_2 - a) - v * v * v * u_2;
            let z = v * v * v * w;
            Ok(Point::new(self.field.div(x, z)?, self.field.div(y, z)?, self.field.div(z, z)?))
        }
    }

    pub fn esm(&self, point: Point, mut scalar: i64) -> anyhow::Result<Point> {
        if !self.is_point_on_curve(point) { anyhow::bail!("Point is not on the curve"); }
        if scalar == 0 { return Ok(Point::new(0, 1, 0)); }
        let mut result = Point::new(0, 1, 0);
        let mut base = point;
        while scalar > 0 {
            if scalar & 1 == 1 { result = self.projective_add(result, base)?; }
            base = self.projective_add(base, base)?;
            scalar >>= 1;
        }
        Ok(result)
    }
}

## Summary

---
In this chapter, I've learned about Elliptic Curves definitions, properties, and operations.

### Exercise 58

> Consider the curve $E_{1,1}(\mathbb{F}_5)$ from example 70 and compute the set of all curve points $(x,y) \in E_{1,1}(\mathbb{F}_5)$.

These two exercises have their own solution in the example.

---

### Exercise 59

> Consider the curve $\texttt{TJJ}\_13$ from example 71 and compute the set of all curve points $(x,y) \in \texttt{TJJ}\_13$.

**📝 Note:** TODO: Implement solution
---

### Exercise 60

> Look up the definition of curve $\texttt{BLS12-381}$, implement it in Sage, and compute the number of all curve points.

**📝 Note:** TODO: Implement solution
---

### Exercise 61

> Let $\mathbb{F}$ be a finite field, let $(a,b)$ and $(a',b')$ be two pairs of parameters, and let $c \in \mathbb{F}^*$ be an invertible field element such that $a' = a \cdot c^4$ and $b' = b \cdot c^6$ hold. Show that the function $I$ from (5.3) maps curve points onto curve points.

Consider a curve like this: $y^2=x^3+a\cdot x+b$. <br>
Assume that $x'=c^2\cdot x$ and $y'=c^3\cdot y$. We have $x'^3=c^6\cdot x^3$ and $y'^2=c^6\cdot y^2$. <br>
$(x',y')$ on the curve, so that $y'^2=x'^3+a'\cdot x'+b'$. <br>
$\Rightarrow c^6\cdot y^2=c^6\cdot x^3 +a\cdot c^4\cdot c^2\cdot x+b\cdot c^6$ <br>
Because $c$ is an invertible field, multiple both side of equation with ${c^{-1}}^6=c^{-6}$: <br>
$\Rightarrow y^2=x^3 +a\cdot x+b$ is a curve. <br>

---

### Exercise 62

> Consider the Tiny-jubjub curve from example 71 and the elliptic curve $E_{7,5}(\mathbb{F}_{13})$ defined as follows: $E_{7,5}(\mathbb{F}_{13}) = \{(x,y) \in \mathbb{F}_{13} \times \mathbb{F}_{13} | y^2 = x^3 + 7x + 5\}$. Show that $TJJ\_13$ and $E_{7,5}(\mathbb{F}_{13})$ are isomorphic. Then compute the set of all points from $E_{7,5}(\mathbb{F}_{13})$, construct $I$ and map all points of $TJJ\_13$ onto $E_{7,5}(\mathbb{F}_{13})$.

$TTJ_{13}: y^2=x^3+8x+8$ and $E_{7,5}(F_{13}): y^2=x^3+7x+5$. <br>
Assume this two curves are isomorphic, that mean $a'=a\cdot c^4$ and $b'=b\cdot c^6$. Base on multiplication table of $13$, we calculate the value of $c^4=\frac{7}{8}=9$ and $c^6=\frac{5}{8}=12$. <br>
Now $c^2=\frac{c^6}{c^4}=10$, and $10$ has square root $\lbrace6,7\rbrace$, that mean there is a possible value for $c$ holds that $a'=a\cdot c^4$ and $b'=b\cdot c^6$.

---

### Exercise 63

> Consider the commutative group $(TJJ\_13, \oplus)$ of the Tiny-jubjub curve from example 71.
> 1. Compute the inverse of $(10,10), \mathcal{O}, (4,0)$ and $(1,2)$.
> 2. Solve the equation $x \oplus (9,4) = (5,2)$ for some $x \in TJJ\_13$.

Calculate the inverses:

- $-(10,10)=(10,3)$
- $-\mathcal{O}=\mathcal{O}$
- $-(4,0)=(4,0)$
- $-(1,2)=(1,11)$

Solve the equation: $x\oplus(9,4)=(5,2)$.

- Plus both side with $-(9,4)=(9,9)$: $x=(5,2)\oplus(9,9)$.
- $x_3=(\frac{y_2-y_1}{x_2-x_1})^2-x_1-x_2=(\frac{9-2}{9-5})^2-5-9=11$
- $y_3=(\frac{y_2-y_1}{x_2-x_1})(x_1-x_3)-y_1=(\frac{9-2}{9-5})(5-11)-9=7$
- $x=(11,7)$

---

### Exercise 64

> Consider example 79 and compute the set $\{(0,1),(0,1), \dots,(0,1),(0,1)\}$ using the tangent rule only.

$$
\begin{align*}
[1](0, 1) + [1](0, 1) = [2](0, 1) \\
[2](0, 1) + [2](0, 1) = [4](0, 1) \\
[4](0, 1) + [4](0, 1) = [8](0, 1) \\
[8](0, 1) + [8](0, 1) = [7](0, 1) \\
[7](0, 1) + [7](0, 1) = [5](0, 1) \\
[5](0, 1) + [5](0, 1) = [1](0, 1)
\end{align*}
$$

$[3]$, $[6]$, $[9]$ is belonged to logarithmic order. And factorization of 9 is $3\cdot 3$, so $[9]$ has 3 subgroups:
- A subgroup of order of 9
- A subgroup of order of 3 contains $[3](0, 1) $ and $[6](0, 1) $
- A subgroup of order of 1

---

### Exercise 65

> Consider example 80 and compute the scalar multiplications $(5,11)$ as well as $(9,4)$ and $(9,4)$ with pen and paper using the algorithm from exercise 38.

In [ ]:
let tini_jubjub = EllipticCurve::new(
    CurveForm::ShortWeierstrass,
    Modular(13),
    8,
    8
);

let point1 = tini_jubjub.esm(Point::new(5, 11, 1), 10).unwrap();
let point2 = tini_jubjub.esm(Point::new(9, 4, 1), 10).unwrap();
let point3 = tini_jubjub.esm(Point::new(9, 4, 1), 4).unwrap();

println!("[10](5, 11) = ({:}, {:})", point1.x, point1.y);
println!("[10](9, 4) = ({:}, {:})", point2.x, point2.y);
println!("[4](9, 4) = ({:}, {:})", point3.x, point3.y);

---

### Exercise 66

> Consider example 81 and compute the set (5.23) by inserting all points from the projective plane $\mathbb{F}_5\mathbb{P}^2$ into the defining projective Short Weierstrass equation.

This exercise is already solved in the example.

---

### Exercise 67

> Compute the projective representation of the Tiny-jubjub curve (example 71) and the logarithmic order of its large prime-order subgroup with respect to the generator $[7 : 11 : 1]$ in projective coordinates.

In [ ]:
let e = EllipticCurve::new(CurveForm::ShortWeierstrass, Modular(13), 8, 8);
let mut logarithm_order: Vec<Point> = Vec::new();

let mut point = Point::new(0, 1, 0);
while e.is_point_on_curve(point) {
    let generator = Point::new(7, 11, 1);
    point = e.projective_add(generator, point).unwrap();
    logarithm_order.push(point);
    if point.is_infinity() {
        break;
    }
}

println!("Logarithm order: {:?}", logarithm_order);

---

### Exercise 68

> Consider example 81 again. Compute the following expression for projective points on $E_{1,1}(\mathbb{F}_5\mathbb{P}^2)$ using algorithm 7:
>   *   $[0 : 1 : 0] \oplus [4 : 3 : 1]$
>   *   $[0 : 3 : 0] \oplus [3 : 1 : 2]$
>   *   $-[0 : 4 : 1] \oplus [3 : 4 : 1]$
>   *   $[4 : 3 : 1] \oplus [4 : 2 : 1]$
>       and then solve the equation $[X : Y : Z] \oplus [0 : 1 : 1] = [2 : 4 : 1]$ for some point $[X : Y : Z]$ from the projective Short Weierstrass curve $E_{1,1}(\mathbb{F}_5\mathbb{P}^2)$.

In [ ]:
let e = EllipticCurve::new(CurveForm::ShortWeierstrass, Modular(5), 1, 1);
println!("[0, 1, 0] + [4, 3, 1] = {:?}", e.projective_add(Point::new(0, 1, 0), Point::new(4, 3, 1)));
println!("[0, 3, 0] + [3, 1, 2] = {:?}", e.projective_add(Point::new(0, 3, 0), Point::new(3, 1, 2)));
println!("-[0, 4, 1] + [3, 4, 1] = {:?}", e.projective_add(e.inverse(Point::new(0, 4, 1)).unwrap(), Point::new(3, 4, 1)));
println!("[4, 3, 1] + [4, 2, 1] = {:?}", e.projective_add(Point::new(4, 3, 1), Point::new(4, 2, 1)));

---

### Exercise 69

> Compare the affine addition law for Short Weierstrass curves with the projective addition rule. Which branch in the projective rule corresponds to which case in the affine law?

**📝 Note:** TODO: Implement solution
---

### Exercise 70

> Consider example 82 and compute the set in (5.30) by inserting every pair of field elements $(x,y) \in \mathbb{F}_{13} \times \mathbb{F}_{13}$ into the defining Montgomery equation.

**📝 Note:** TODO: Implement solution
---

### Exercise 71

> Consider the elliptic curve $E_{1,1}(\mathbb{F}_5)$ from example 70 and show that $E_{1,1}(\mathbb{F}_5)$ is not a Montgomery curve.

**📝 Note:** TODO: Implement solution
---

### Exercise 72

> Consider the elliptic curve secp256k1 from example 72 and show that secp256k1 is not a Montgomery curve.

**📝 Note:** TODO: Implement solution
---

### Exercise 73

> Consider the commutative group $(\text{M\_TJJ\_13}, \oplus)$ of the Tiny-jubjub curve in its Montgomery form from example (5.30).
> 1. Compute the inverse of $(1,9), \mathcal{O}, (7,12)$ and $(4,9)$.
> 2. Solve the equation $x \oplus (3,8) = (10,3)$ for some $x \in \text{M\_TJJ\_13}$.
> Choose some element $x \in \text{M\_TJJ\_13}$ and test if $x$ is a generator of $\text{M\_TJJ\_13}$. If $x$ is not a generator, repeat until you find some generator $x$. Write $\text{M\_TJJ\_13}$ in logarithmic order with respect to $x$.

**📝 Note:** TODO: Implement solution
---

### Exercise 74

> Consider the curve alt_bn128 from example 73. Show that this curve is not a Montgomery curve.

**📝 Note:** TODO: Implement solution
---

### Exercise 75

> Consider the commutative group $(\text{TE\_TJJ\_13}, \oplus)$ from example 85.
> 1. Compute the inverse of $(1,11), (0,1), (3,0)$ and $(5,8)$.
> 2. Solve the equation $x \oplus (5,8) = (1,11)$ for some $x \in \text{TE\_TJJ\_13}$.
> Choose some element $x \in \text{TE\_TJJ\_13}$, and test if $x$ is a generator of $\text{TE\_TJJ\_13}$. If $x$ is not a generator, repeat until you find some generator $x$. Write $\text{TE\_TJJ\_13}$ in logarithmic order with respect to $x$.

**📝 Note:** TODO: Implement solution
---

### Exercise 76

> Consider the Short Weierstrass elliptic curve $E(\mathbb{F}_{5^2})$ from example 90, compute the expression $(4t + 3, 2t + 1) \oplus (3t + 3, 2)$ using pen and paper and double-check the computation using sage. Then solve the equation $x \oplus (3t + 3, 3) = (3, 4)$ for some $x \in E(\mathbb{F}_{5^2})$. After that compute the scalar multiplication $(2t + 1, 4t + 4)$ using the double-and-add algorithm from exercise 38.

**📝 Note:** TODO: Implement solution
---

### Exercise 77

> Consider the Tiny-jubjub curve from example 71. Show that the polynomial $t^4 + 2 \in \mathbb{F}_{13}[t]$ is irreducible. Then write a sage program to implement the finite field extension $\mathbb{F}_{13^4}$, implement the curve extension $TJJ\_13(\mathbb{F}_{13^4})$ and compute the number of curve points.

**📝 Note:** TODO: Implement solution
---

### Exercise 78

> Consider the alt_bn128 curve and its associated base field $\mathbb{F}_p$ from example 73. As we know from example 89 this curve has an embedding degree of 12. Use Sage to find an irreducible polynomial $P \in \mathbb{F}_p[t]$ and write a sage program to implement the finite field extension $\mathbb{F}_{p^{12}}$ and to implement the curve extension alt\_bn128($\mathbb{F}_{p^{12}}$) and compute the number of curve points.

**📝 Note:** TODO: Implement solution
---

### Exercise 79

> Consider the full 5-torsion group $TJJ\_13$ from example 92. Write down the set of all elements from this group and identify the subset of all elements from $TJJ\_13(\mathbb{F}_{13})$ as well as $TJJ\_13(\mathbb{F}_{13^2})$. Then compute the 5-torsion group $TJJ\_13(\mathbb{F}_{13^8})$.

**📝 Note:** TODO: Implement solution
---

### Exercise 80

> Consider the curve secp256k1 from example 72 and its full r-torsion group as introduced in example 93. Write down a single element from the curves full torsion group that is not the point at infinity.

**📝 Note:** TODO: Implement solution
---

### Exercise 81

> Consider the curve alt_bn128 from example 73 and its curve extension from exercise 78. Write a Sage program that computes a generator from the curves full torsion group.

**📝 Note:** TODO: Implement solution
---

### Exercise 82

> Consider the small prime factor 2 of the Tiny-jubjub curve. Compute the full 2-torsion group of $TJJ\_13$ and then compute the groups $G_1$ and $G_2$.

**📝 Note:** TODO: Implement solution
---

### Exercise 83

> Consider the curve alt_bn128 from example 73 and its curve extension from exercise 78. Write a Sage program that computes a generator for each of the torsion group $G_1[p]$ and $G_2[p]$.

**📝 Note:** TODO: Implement solution
---

### Exercise 84

> Consider the curve alt_bn128 from example 73 and the generators $g_1$ and $g_2$ of $G_1[p]$ and $G_2[p]$ from exercise 83. Write a Sage program that computes the Weil pairing $e(g_1, g_2)$.

**📝 Note:** TODO: Implement solution
---

### Exercise 85

> Use our definition of the try\_hash algorithm to implement a hash function $H_{TJJ\_13} : \{0,1\}^* \to TJJ\_13(\mathbb{F}_{13})$ that maps binary strings of arbitrary length onto the 5-torsion group of $TJJ13(\mathbb{F}_{13})$.

**📝 Note:** TODO: Implement solution
---

### Exercise 86

> Implement a cryptographic hash function $H_{secp256k1} : \{0,1\}^* \to secp256k1$ that maps binary strings of arbitrary length onto the elliptic curve secp256k1.

**📝 Note:** TODO: Implement solution
---

### Exercise 87

> Consider the curve alt_bn128 from example 73. Write a Sage program that computes the trace of Frobenius for alt_bn128. Does the curve contain more or less elements than its base field $\mathbb{F}_p$?

**📝 Note:** TODO: Implement solution
---

### Exercise 88

> Consider the curve alt\_bn128 from example 73. Write a Sage program that computes the j-invariant for alt\_bn128.

**📝 Note:** TODO: Implement solution
---

### Exercise 89

> Show that the Hilbert class polynomials for the CM-discriminants $D = -3$ and $D = -4$ are given by $H_{-3,q}(x) = x$ and $H_{-4,q} = x - (1728 \pmod q)$.

**📝 Note:** TODO: Implement solution
---

### Exercise 90

> Use the complex multiplication method to construct an elliptic curve of order 7 over the prime field $\mathbb{F}_{13}$.

**📝 Note:** TODO: Implement solution
---

### Exercise 91

> Use the complex multiplication method to compute all isomorphism classes of all elliptic curves of order 7 over the prime field $\mathbb{F}_{13}$.

**📝 Note:** TODO: Implement solution
---

### Exercise 92

> Consider the prime modulus $p$ of curve alt_bn128 from example 73 and its trace $t$ from exercise 92. Use the complex multiplication method to synthesize an elliptic curve over $\mathbb{F}_p$ that is isomorphic to alt_bn128 and compute an explicit isomorphism between these two curves.

**📝 Note:** TODO: Implement solution
---

### Exercise 93

> Consider the point $P = (9,2)$. Show that $P$ is a point on the BLS6_6 curve and compute the scalar product $P$.

> [!CAUTION] <br>
> No solution yet.
---

### Exercise 94

> Compute the following expressions: $-(26,34), (26,9) \oplus (13,28), (35,15) \oplus \mathcal{O}$ and $(27,9) \ominus (33,9)$.

**📝 Note:** TODO: Implement solution
---

### Exercise 95

> Consider the extended BLS6_6 curve as defined in 5.67 and the two curve points $g_1 = (13,15)$ and $g_2 = (7v^2, 16v^3)$. Compute the Weil pairing $e(g_1, g_2)$ using definition 5.49 and Miller’s algorithm 8.

**📝 Note:** TODO: Implement solution
---